In [642]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [643]:
df = pd.read_csv('../../python/csv_files/Bengaluru_House_Data.csv')
df

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00
...,...,...,...,...,...,...,...,...,...
13315,Built-up Area,Ready To Move,Whitefield,5 Bedroom,ArsiaEx,3453,4.0,0.0,231.00
13316,Super built-up Area,Ready To Move,Richards Town,4 BHK,NaN,3600,5.0,NaN,400.00
13317,Built-up Area,Ready To Move,Raja Rajeshwari Nagar,2 BHK,Mahla T,1141,2.0,1.0,60.00
13318,Super built-up Area,18-Jun,Padmanabhanagar,4 BHK,SollyCl,4689,4.0,1.0,488.00


# Data cleaning

In [644]:
df.isna().sum()

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

In [645]:
df.groupby('area_type')['area_type'].count()

area_type
Built-up  Area          2418
Carpet  Area              87
Plot  Area              2025
Super built-up  Area    8790
Name: area_type, dtype: int64

In [646]:
df = df.drop(['area_type', 'society', 'balcony', 'availability'], axis='columns')
df

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00
...,...,...,...,...,...
13315,Whitefield,5 Bedroom,3453,4.0,231.00
13316,Richards Town,4 BHK,3600,5.0,400.00
13317,Raja Rajeshwari Nagar,2 BHK,1141,2.0,60.00
13318,Padmanabhanagar,4 BHK,4689,4.0,488.00


In [647]:
df.isna().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [648]:
df.bath = df.bath.fillna(df.bath.median())
df.dropna(inplace=True)
df.isna().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

In [649]:
df['size'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [650]:
df['BHK'] = df['size'].str.split(' ', expand=True)[0].astype(int)
df.head()

,location,size,total_sqft,bath,price,BHK
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2


In [651]:
df['BHK'].unique()

array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 27, 10, 19, 16, 43, 14, 12,
       13, 18])

In [652]:
df[df.BHK > 20]

,location,size,total_sqft,bath,price,BHK
1718,2Electronic City Phase II,27 BHK,8000,27.0,230.0,27
4684,Munnekollal,43 Bedroom,2400,40.0,660.0,43


In [653]:
df.total_sqft.unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [654]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [655]:
df[~df['total_sqft'].apply(is_float)].head(10)

,location,size,total_sqft,bath,price,BHK
30,Yelahanka,4 BHK,2100 - 2850,4.0,186.000,4
56,Devanahalli,4 Bedroom,3010 - 3410,2.0,192.000,4
81,Hennur Road,4 Bedroom,2957 - 3450,2.0,224.500,4
122,Hebbal,4 BHK,3067 - 8156,4.0,477.000,4
137,8th Phase JP Nagar,2 BHK,1042 - 1105,2.0,54.005,2
165,Sarjapur,2 BHK,1145 - 1340,2.0,43.490,2
188,KR Puram,2 BHK,1015 - 1540,2.0,56.800,2
224,Devanahalli,3 BHK,1520 - 1740,2.0,74.820,3
410,Kengeri,1 BHK,34.46Sq. Meter,1.0,18.500,1
549,Hennur Road,2 BHK,1195 - 1440,2.0,63.770,2


In [656]:
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0]) + float(tokens[1])) / 2
    try:
        return float(x)
    except:
        return x

In [657]:
df['total_sqft'] = df['total_sqft'].apply(convert_sqft_to_num)
df[~df['total_sqft'].apply(is_float)]

,location,size,total_sqft,bath,price,BHK
410,Kengeri,1 BHK,34.46Sq. Meter,1.0,18.50,1
648,Arekere,9 Bedroom,4125Perch,9.0,265.00,9
775,Basavanagara,1 BHK,1000Sq. Meter,2.0,93.00,1
872,Singapura Village,2 BHK,1100Sq. Yards,2.0,45.00,2
1019,Marathi Layout,1 Bedroom,5.31Acres,1.0,110.00,1
1086,Narasapura,2 Bedroom,30Acres,2.0,29.50,2
1400,Chamrajpet,9 BHK,716Sq. Meter,9.0,296.00,9
1712,Singena Agrahara,3 Bedroom,1500Sq. Meter,3.0,95.00,3
1743,Hosa Road,3 BHK,142.61Sq. Meter,3.0,115.00,3
1821,Sarjapur,3 Bedroom,1574Sq. Yards,3.0,76.00,3


In [658]:
def convert_rest(x):
    if is_float(x):
        return float(x)
    match = re.match(r'(\d+\.?\d*)\s+([a-zA-Z]+)', x)
    if match:
        num = float(match.group(1))
        unit = match.group(2)
        if unit == 'Sq. Meter':
            return num * 10.764
        elif unit == 'Acres':
            return num * 43560
        elif unit == 'Sq. Yards':
            return num * 9
        elif unit == 'Perch':
            return num * 272.3
        elif unit == 'Cents':
            return num * 435.56

In [659]:
df['total_sqft' ] = df['total_sqft'].apply(convert_rest)
df[~df['total_sqft'].apply(is_float)]

,location,size,total_sqft,bath,price,BHK


In [660]:
df.head(10)

,location,size,total_sqft,bath,price,BHK
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3
4,Kothanur,2 BHK,1200.0,2.0,51.00,2
5,Whitefield,2 BHK,1170.0,2.0,38.00,2
6,Old Airport Road,4 BHK,2732.0,4.0,204.00,4
7,Rajaji Nagar,4 BHK,3300.0,4.0,600.00,4
8,Marathahalli,3 BHK,1310.0,3.0,63.25,3
9,Gandhi Bazar,6 Bedroom,1020.0,6.0,370.00,6


In [661]:
df.dropna(inplace=True)

# Feature engineering

In [663]:
# Convert sqft to sqmt
df['total_sqmt'] = df['total_sqft'] * 10.7639
df.drop(['total_sqft'], axis='columns', inplace=True)
df

,location,size,bath,price,BHK,total_sqmt
0,Electronic City Phase II,2 BHK,2.0,39.07,2,11366.6784
1,Chikka Tirupathi,4 Bedroom,5.0,120.00,4,27986.1400
2,Uttarahalli,3 BHK,2.0,62.00,3,15500.0160
3,Lingadheeranahalli,3 BHK,3.0,95.00,3,16371.8919
4,Kothanur,2 BHK,2.0,51.00,2,12916.6800
...,...,...,...,...,...,...
13315,Whitefield,5 Bedroom,4.0,231.00,5,37167.7467
13316,Richards Town,4 BHK,5.0,400.00,4,38750.0400
13317,Raja Rajeshwari Nagar,2 BHK,2.0,60.00,2,12281.6099
13318,Padmanabhanagar,4 BHK,4.0,488.00,4,50471.9271


In [664]:
# Convert Lakh Rupees to Euros
df['price'] = df['price'] * 100000 * 0.011
df

,location,size,bath,price,BHK,total_sqmt
0,Electronic City Phase II,2 BHK,2.0,42977.0,2,11366.6784
1,Chikka Tirupathi,4 Bedroom,5.0,132000.0,4,27986.1400
2,Uttarahalli,3 BHK,2.0,68200.0,3,15500.0160
3,Lingadheeranahalli,3 BHK,3.0,104500.0,3,16371.8919
4,Kothanur,2 BHK,2.0,56100.0,2,12916.6800
...,...,...,...,...,...,...
13315,Whitefield,5 Bedroom,4.0,254100.0,5,37167.7467
13316,Richards Town,4 BHK,5.0,440000.0,4,38750.0400
13317,Raja Rajeshwari Nagar,2 BHK,2.0,66000.0,2,12281.6099
13318,Padmanabhanagar,4 BHK,4.0,536800.0,4,50471.9271


In [665]:
df['price_per_sqmt'] = df['price'] / df['total_sqmt']
df.head(10)

,location,size,bath,price,BHK,total_sqmt,price_per_sqmt
0,Electronic City Phase II,2 BHK,2.0,42977.0,2,11366.6784,3.780964
1,Chikka Tirupathi,4 Bedroom,5.0,132000.0,4,27986.1400,4.716620
2,Uttarahalli,3 BHK,2.0,68200.0,3,15500.0160,4.399995
3,Lingadheeranahalli,3 BHK,3.0,104500.0,3,16371.8919,6.382891
4,Kothanur,2 BHK,2.0,56100.0,2,12916.6800,4.343221
5,Whitefield,2 BHK,2.0,41800.0,2,12593.7630,3.319103
6,Old Airport Road,4 BHK,4.0,224400.0,4,29406.9748,7.630843
7,Rajaji Nagar,4 BHK,4.0,660000.0,4,35520.8700,18.580626
8,Marathahalli,3 BHK,3.0,69575.0,3,14100.7090,4.934149
9,Gandhi Bazar,6 Bedroom,6.0,407000.0,6,10979.1780,37.070170


In [666]:
df['location'].nunique()

1298

In [667]:
df.location = df.location.apply(lambda x: x.strip())
location_stats = df.groupby('location')['location'].count()
location_stats.sort_values(ascending=False).head(20)

location
Whitefield                  538
Sarjapur  Road              397
Electronic City             304
Kanakpura Road              271
Thanisandra                 236
Yelahanka                   212
Uttarahalli                 186
Hebbal                      177
Marathahalli                175
Raja Rajeshwari Nagar       171
Hennur Road                 152
Bannerghatta Road           151
7th Phase JP Nagar          148
Haralur Road                142
Electronic City Phase II    132
Rajaji Nagar                107
Chandapura                  100
Bellandur                    96
KR Puram                     91
Electronics City Phase 1     88
Name: location, dtype: int64

In [668]:
df['location'] = df['location'].apply(lambda x: 'other' if location_stats[x] < 10 else x)
df.groupby('location')['location'].count().sort_values(ascending=False).head(20)

location
other                       2733
Whitefield                   538
Sarjapur  Road               397
Electronic City              304
Kanakpura Road               271
Thanisandra                  236
Yelahanka                    212
Uttarahalli                  186
Hebbal                       177
Marathahalli                 175
Raja Rajeshwari Nagar        171
Hennur Road                  152
Bannerghatta Road            151
7th Phase JP Nagar           148
Haralur Road                 142
Electronic City Phase II     132
Rajaji Nagar                 107
Chandapura                   100
Bellandur                     96
KR Puram                      91
Name: location, dtype: int64

# Outlier removal

In [669]:
df[~(df.total_sqmt / df.BHK < 28)]

,location,size,bath,price,BHK,total_sqmt,price_per_sqmt
0,Electronic City Phase II,2 BHK,2.0,42977.0,2,11366.6784,3.780964
1,Chikka Tirupathi,4 Bedroom,5.0,132000.0,4,27986.1400,4.716620
2,Uttarahalli,3 BHK,2.0,68200.0,3,15500.0160,4.399995
3,Lingadheeranahalli,3 BHK,3.0,104500.0,3,16371.8919,6.382891
4,Kothanur,2 BHK,2.0,56100.0,2,12916.6800,4.343221
...,...,...,...,...,...,...,...
13315,Whitefield,5 Bedroom,4.0,254100.0,5,37167.7467,6.836573
13316,other,4 BHK,5.0,440000.0,4,38750.0400,11.354827
13317,Raja Rajeshwari Nagar,2 BHK,2.0,66000.0,2,12281.6099,5.373888
13318,Padmanabhanagar,4 BHK,4.0,536800.0,4,50471.9271,10.635615


In [670]:
df = df[~(df.total_sqmt / df.BHK < 28)]
df

,location,size,bath,price,BHK,total_sqmt,price_per_sqmt
0,Electronic City Phase II,2 BHK,2.0,42977.0,2,11366.6784,3.780964
1,Chikka Tirupathi,4 Bedroom,5.0,132000.0,4,27986.1400,4.716620
2,Uttarahalli,3 BHK,2.0,68200.0,3,15500.0160,4.399995
3,Lingadheeranahalli,3 BHK,3.0,104500.0,3,16371.8919,6.382891
4,Kothanur,2 BHK,2.0,56100.0,2,12916.6800,4.343221
...,...,...,...,...,...,...,...
13315,Whitefield,5 Bedroom,4.0,254100.0,5,37167.7467,6.836573
13316,other,4 BHK,5.0,440000.0,4,38750.0400,11.354827
13317,Raja Rajeshwari Nagar,2 BHK,2.0,66000.0,2,12281.6099,5.373888
13318,Padmanabhanagar,4 BHK,4.0,536800.0,4,50471.9271,10.635615


In [671]:
df.price_per_sqmt.describe()

count    13255.000000
mean         6.985107
std          9.991187
min          0.273705
25%          4.364322
50%          5.557347
75%          7.472864
max        687.483161
Name: price_per_sqmt, dtype: float64